<a href="https://colab.research.google.com/github/GabMaffei/Trabalho_M2_Grafos_Coloracao/blob/main/Trabalho_M2_Grafos_(Colora%C3%A7%C3%A3o).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classe Grafos

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
class Grafos:
  def __init__(self, direcionado = False, ponderado = False):
    self.ponderado = ponderado
    self.direcionado = direcionado

    self.vertices = 0
    self.arestas = 0

# GrafoMatriz

In [4]:
class GrafoMatriz(Grafos):
  def __init__(self, direcionado = False, ponderado = False):
    super().__init__(direcionado, ponderado)
    self.matriz = list()
    self.labels = dict() #Contém os nomes de vértices

  def __str__(self): #imprimeGrafo
    return (str(self.matriz))

  def _balanceamentoVerticeRemover(self, idVertice):
    #Ajusta o comprimento do vértice, após remoções
    matrizTmp = list()
    for verticeAtual in self.matriz:
      verticeTmp = list()
      for idx, aresta in enumerate(verticeAtual):
        if idx == idVertice:
          continue
        verticeTmp.append(aresta)
      matrizTmp.append(verticeTmp)
    self.matriz = matrizTmp

  def _balanceamentoMatriz(self):
    #Ajusta o comprimento da matriz
    arestasDaVerticeAtual = list()
    for vertice in range(self.vertices - 1):
      arestasDaVerticeAtual = self.matriz[vertice]
      arestasDaVerticeAtual.append([0, 0])
      self.matriz[vertice] = arestasDaVerticeAtual

  def inserirVertice(self, label):
    #Adiciona um vértice sem nenhuma aresta associada a ele, pode parecer 
    # igual em ambos os casos, mas não é.
    #Precisamos adicionar esse vértice no vetor de vértices e também alocar
    # seu espaço para as arestas.

    #Define nome da vértice
    self.labels[label] = self.vertices
    self.vertices += 1

    #Cria linha com arestas para a nova vértice
    arestasDaVerticeAtual = list()

    for vertice in range(self.vertices):
      arestasDaVerticeAtual.append([0, 0])

    self.matriz.append(arestasDaVerticeAtual)

    if self.vertices > 1:
      self._balanceamentoMatriz()
  
  def removerVertice(self, label):
    #Remove um vértice do grafo, elimina a linha e coluna dele da matriz e a 
    # referência dele da lista, junto com todas as arestas que chegam e saem 
    # dele.
    #Converte o indice para label
    if isinstance(label, int):
      label = self.labelVertice(label)
    
    #Removendo o label
    idVertice = self.labels.get(label)
    self.labels.pop(label)
    #Balanceando os labels
    labelsTmp = dict()
    count = 0
    for label in self.labels.items():
      labelsTmp[label[0]] = count
      count += 1
    self.labels = labelsTmp
    #Diminuindo o número de vértices
    self.vertices -= 1

    matrizTmp = list()
    for vertice in range(self.vertices + 1):
      if vertice == idVertice:
        continue
      matrizTmp.append(self.matriz[vertice])

    self.matriz = matrizTmp

    #Balanceamento Negativo
    self._balanceamentoVerticeRemover(idVertice)

  def labelVertice(self, indice):
    #Funções básicas para retornar o nome de um vértice.
    # return str(self.labels.get(indice))
    label = [k for k, v in self.labels.items() if v == indice]
    return (label[0])

  def imprimeGrafo(self):
    #Imprimir o grafo no console, tentem deixar próximo da representação 
    # dos slides (não precisa da grade da matriz).
    print('Grafo:')
    print('   ' + str(list(range(len(self.matriz)))))
    for idx, vertice in enumerate(self.matriz):
      print(str(idx) + ': ' + str(vertice))

  def inserirAresta(self, origem, destino, peso = 1): 
    #Essa operação deve ter um cuidado especial,
    # ela deve ser executada levando em conta o tipo do grafo.
    # No caso de um grafo ponderado o peso deve ser aplicado e
    # no caso de um grafo direcionado, uma ligação de volta deve ser adicionada;
    
    #Converte os labels para indices
    if isinstance(origem, str):
      origem = self.labels.get(origem)
    if isinstance(destino, str):
      destino = self.labels.get(destino)
    #Se não ponderado, o peso é igual a 1
    if not self.ponderado:
      peso = 1

    #Adiciona a ligação
    arestaTmp = list()
    for idx, aresta in enumerate(self.matriz[origem]):
      if idx == destino:
        arestaTmp.append([1, peso])
        continue
      arestaTmp.append(aresta)
    self.matriz[origem] = arestaTmp
    
    #Se direcionado, adiciona a ligação de volta
    if self.direcionado:
      arestaTmp = list()
      for idx, aresta in enumerate(self.matriz[destino]):
        if idx == origem:
          arestaTmp.append([1, peso])
          continue
        arestaTmp.append(aresta)
      self.matriz[destino] = arestaTmp

  def removerAresta(self, origem, destino):
    #Remove uma aresta entre dois vértices no grafo, lembrando 
    # que no grafo não direcionado deve ser removida a aresta de retorno também;
    #Converte os labels para indices
    if isinstance(origem, str):
      origem = self.labels.get(origem)
    if isinstance(destino, str):
      destino = self.labels.get(destino)

    #Remove a ligação
    arestaTmp = list()
    for idx, aresta in enumerate(self.matriz[origem]):
      if idx == destino:
        arestaTmp.append([0, 0])
        continue
      arestaTmp.append(aresta)
    self.matriz[origem] = arestaTmp
    
    #Se direcionado, remove a ligação de volta
    if self.direcionado:
      arestaTmp = list()
      for idx, aresta in enumerate(self.matriz[destino]):
        if idx == origem:
          arestaTmp.append([0, 0])
          continue
        arestaTmp.append(aresta)
      self.matriz[destino] = arestaTmp

  def existeAresta(self, origem, destino):
    #Verifica a existência de uma aresta, aqui vemos uma diferença 
    # grande entre matriz e lista.
    #Converte os labels para indices
    if isinstance(origem, str):
      origem = self.labels.get(origem)
    if isinstance(destino, str):
      destino = self.labels.get(destino)

    #for idx, aresta in enumerate(self.matriz[origem]):
    #  if idx == destino and aresta[0] == 1:
    #    return True
    
    if self.matriz[origem][destino][0] == 1:
      return True

    return False

  def pesoAresta(self, origem, destino):
    #Retorne o peso de uma aresta, aqui vemos uma diferença
    # grande entre matriz e lista.
    #Converte os labels para indices
    if isinstance(origem, str):
      origem = self.labels.get(origem)
    if isinstance(destino, str):
      destino = self.labels.get(destino)
    
    if self.ponderado == False:
      return 0

    for idx, aresta in enumerate(self.matriz[origem]):
      if idx == destino:
        return aresta[1]

    return 0

  def retornarVizinhos(self, vertice):
    #Função para retorno dos vizinhos de um vértice, necessária pois não 
    # teremos acesso a estrutura das arestas para os próximos algoritmos.
    #Converte os labels para indices
    if isinstance(vertice, str):
      vertice = self.labels.get(vertice)

    #Só verifica os vizinhos da vértice, ou seja, não verifica se existem
    # arestas ligadas a vértice atual com origem em outras vértices
    # Dúvida: preciso retornar os antecessores?
    # https://www.inf.ufsc.br/grafos/definicoes/definicao.html#:~:text=ADJAC%C3%8ANCIA,de%20Antonio.
    vizinhosIdx = list()
    for idx, aresta in enumerate(self.matriz[vertice]):
      if aresta[0] == 1:
        tuplaTemp = (self.labelVertice(idx), idx)
        vizinhosIdx.append(tuplaTemp)
    return vizinhosIdx
'''
grafoExemplo = GrafoMatriz(direcionado = False, ponderado = False)
grafoExemplo.inserirVertice('um')
grafoExemplo.inserirVertice('dois')
grafoExemplo.inserirVertice('tres')
# print(grafoExemplo)

# print(grafoExemplo.labelVertice(0))
# print(grafoExemplo.labelVertice(1))

# print(grafoExemplo)

# print(grafoExemplo.labelVertice(0))
# print(grafoExemplo.labelVertice(1))

grafoExemplo.imprimeGrafo()

grafoExemplo.inserirAresta(0, 1)
grafoExemplo.removerVertice('tres')
grafoExemplo.imprimeGrafo()
print(grafoExemplo.existeAresta(0, 1))
print(grafoExemplo.existeAresta(1, 0))
# print(grafoExemplo.pesoAresta(0, 1))
# print(grafoExemplo.pesoAresta(1, 0))
print(grafoExemplo.retornarVizinhos(0))
print(grafoExemplo.retornarVizinhos(1))

grafoExemplo.removerAresta(0, 1)
grafoExemplo.imprimeGrafo()
# print(grafoExemplo.existeAresta(0, 1))
# print(grafoExemplo.existeAresta(1, 0))
# print(grafoExemplo.pesoAresta(0, 1))
# print(grafoExemplo.pesoAresta(1, 0))
print(grafoExemplo.retornarVizinhos(0))
print(grafoExemplo.retornarVizinhos(1))
'''

"\ngrafoExemplo = GrafoMatriz(direcionado = False, ponderado = False)\ngrafoExemplo.inserirVertice('um')\ngrafoExemplo.inserirVertice('dois')\ngrafoExemplo.inserirVertice('tres')\n# print(grafoExemplo)\n\n# print(grafoExemplo.labelVertice(0))\n# print(grafoExemplo.labelVertice(1))\n\n# print(grafoExemplo)\n\n# print(grafoExemplo.labelVertice(0))\n# print(grafoExemplo.labelVertice(1))\n\ngrafoExemplo.imprimeGrafo()\n\ngrafoExemplo.inserirAresta(0, 1)\ngrafoExemplo.removerVertice('tres')\ngrafoExemplo.imprimeGrafo()\nprint(grafoExemplo.existeAresta(0, 1))\nprint(grafoExemplo.existeAresta(1, 0))\n# print(grafoExemplo.pesoAresta(0, 1))\n# print(grafoExemplo.pesoAresta(1, 0))\nprint(grafoExemplo.retornarVizinhos(0))\nprint(grafoExemplo.retornarVizinhos(1))\n\ngrafoExemplo.removerAresta(0, 1)\ngrafoExemplo.imprimeGrafo()\n# print(grafoExemplo.existeAresta(0, 1))\n# print(grafoExemplo.existeAresta(1, 0))\n# print(grafoExemplo.pesoAresta(0, 1))\n# print(grafoExemplo.pesoAresta(1, 0))\nprint(gr

# GrafoLista

In [86]:
from typing_extensions import Self
from operator import itemgetter
from queue import PriorityQueue
import time
class GrafoLista(Grafos):

  def __init__(self, direcionado = False, ponderado= False):
    super().__init__(direcionado, ponderado) # Direcionado = False significa que é direcionado, e True que não é Direcionado
    self.lista = list()
    self.labels = dict() #Contém os nomes de vértices
    self.listaColorida = dict() #Contém os labels e sua cor correspondente

    self.vertices = 0
    self.arestas = 0
    self.cores = 0 # Tem uma cor / Cada cor aumenta em 1 esse número, e adiciona a cor correspondente ao número anterior
    # Exemplo: cores = 3, existem 4 cores, sendo 0,1,2,3

  def __str__(self):
    return (str(self.lista))

  def openFile(self, filename):
    if self.vertices != 0 or self.arestas != 0:
      raise Exception('A instância do objeto GrafoLista não está vazia.')

    with open(filename, 'r') as arquivo:
      verticesArquivo, arestasArquivo, direcionadoArquivo, ponderadoArquivo = (arquivo.readline()).split()
      #Checa se é direcionado
      if int(direcionadoArquivo) == 1:
        self.direcionado = False
      else:
        self.direcionado = True
      #Checa se é ponderado
      if int(ponderadoArquivo) == 1:
        self.ponderado = True
      else:
        self.ponderado = False

      for vertice in range(int(verticesArquivo)):
        self.inserirVertice(str(vertice))

      if self.ponderado:
        for aresta in range(int(arestasArquivo)):
          origem, destino, peso = (arquivo.readline()).split()
          self.inserirAresta(origem, destino, float(peso))
      else:
          for aresta in range(int(arestasArquivo)):
            origem, destino = (arquivo.readline()).split()
            self.inserirAresta(origem, destino)

  def buscaProfunda(self, origem, destino):
    caminho = []
    visitado = set()
    return self._buscaProfunda(origem, destino, caminho, visitado)

  def _buscaProfunda(self, origem, destino, caminho = [], visitado = set()):
    #Converte os labels para indices
    if isinstance(origem, str):
      origem = self.labels.get(origem)
    if isinstance(destino, str):
      destino = self.labels.get(destino)
    caminho.append(origem)
    visitado.add(origem)
    
    if origem == destino:
        return caminho
    for (vizinho, peso) in self.lista[origem]:
        if vizinho not in visitado:
            resultado = self._buscaProfunda(vizinho, destino, caminho, visitado)
            if resultado is not None:
                return resultado
    #caminho.pop() #diferença entre o visitado e o caminho, sem o pop, ele é igual a ordem visitada
    return None

  def buscaLargura(self, origem, destino, validaDestinoAntes = True, visitadoDebug = False):
    #Converte os labels para indices
    if isinstance(origem, str):
      origem = self.labels.get(origem)
    if isinstance(destino, str):
      destino = self.labels.get(destino)

    visitados = [origem]
    fila = [origem]

    while len(fila) > 0:
      # print('Laço de repetição!')
      verticeAtual = fila.pop()

      if verticeAtual == destino:
        # print('verticeAtual == destino' + str(verticeAtual) + str(destino), sep=';')
        if verticeAtual not in visitados:
          visitados.append(verticeAtual)
        return visitados

      vizinhos = self.retornarVizinhos(verticeAtual)
      # print('VerticeAtual: ' + str(verticeAtual), end=' ')
      # print('Vizinhos: ' + str(vizinhos))

      for verticeVizinha in vizinhos:
        idxVerticeVizinha = self.labels.get(verticeVizinha[0])
        # print('idxVerticeVizinha: ' + str(idxVerticeVizinha))
        if idxVerticeVizinha not in visitados:
          fila.append(idxVerticeVizinha)
          visitados.append(idxVerticeVizinha)
          if idxVerticeVizinha == destino and validaDestinoAntes:
            return visitados
      
      # print('Fila: ' + str(fila), end=' ')
      # print('Visitados: ' + str(visitados))
      # print('Length Fila: ' + str(len(fila)))
      
    if visitadoDebug:
      return visitados
    return None

  def buscaDijkstra(self, origem, destino):
    #Converte os labels para indices
    if isinstance(origem, str):
      origem = self.labels.get(origem)
    if isinstance(destino, str):
      destino = self.labels.get(destino)

    caminhosMaisCurtos = self.dijkstra(origem)
    return caminhosMaisCurtos[destino]

  def dijkstra(self, origem):
      #Converte os labels para indices
      if isinstance(origem, str):
        origem = self.labels.get(origem)

      caminhosMaisCurtos = {vertice:float('inf') for vertice in range(self.vertices)}
      caminhosMaisCurtos[origem] = 0
      # print('caminhosMaisCurtos inicial:', caminhosMaisCurtos)

      fila = PriorityQueue()
      fila.put((0, origem))
      # print('Fila inicial:', fila.queue)
      visitado = list()

      while not fila.empty():
          (distanciaAtual, verticeAtual) = fila.get()
          visitado.append(verticeAtual)

          vizinhos = self.retornarVizinhos(verticeAtual)
          # print('Vizinhos:', vizinhos)
          # print('Fila loop:', fila.queue)
          for vizinho in vizinhos:
            # print('verticeAtual:', verticeAtual, end=' ')
            # print('VizinhoAtual:', vizinho, end=' ')
            distanciaVizinho = self.pesoArestaLabel(verticeAtual, vizinho[1])
            # print('distanciaVizinhoAtual:', distanciaVizinho)
            if vizinho not in visitado:
              custoAntigo = caminhosMaisCurtos[self.labels.get(vizinho[0])]
              custoNovo = caminhosMaisCurtos[verticeAtual] + distanciaVizinho
              if custoNovo < custoAntigo:
                fila.put((custoNovo, self.labels.get(vizinho[0])))
                caminhosMaisCurtos[self.labels.get(vizinho[0])] = custoNovo

      return caminhosMaisCurtos

  def _ordernarArestasDoVertice(self):
    for verticeId in range(len(self.lista)):
      self.lista[verticeId].sort(key=itemgetter(0))

  def inserirVertice(self, label):
    #Adiciona um vértice sem nenhuma aresta associada a ele, pode parecer 
    # igual em ambos os casos, mas não é.
    #Precisamos adicionar esse vértice no vetor de vértices e também alocar
    # seu espaço para as arestas.

    #Define nome da vértice
    self.labels[label] = self.vertices
    self.vertices += 1

    self.lista.append(list())
  
  def removerVertice(self, label):
    #Remove um vértice do grafo, elimina a linha e coluna dele da matriz 
    # e a referência dele da lista, junto com todas as arestas que chegam 
    # e saem dele.
    #Converte o indice para label
    if isinstance(label, int):
      label = self.labelVertice(label)

    #Removendo o label
    idVertice = self.labels.get(label)
    self.labels.pop(label)
    #Balanceando os labels
    labelsTmp = dict()
    count = 0
    for label in self.labels.items():
      labelsTmp[label[0]] = count
      count += 1
    self.labels = labelsTmp
    #Diminuindo o número de vértices
    self.vertices -= 1

    listaTmp = list()
    for vertice in range(self.vertices + 1):
      if vertice == idVertice:
        continue
      listaTmp.append(self.lista[vertice])

    self.lista = listaTmp
  
  def labelVertice(self, indice):
    #Funções básicas para retornar o nome de um vértice.
    label = [k for k, v in self.labels.items() if v == indice]
    return (label[0])

  def imprimeGrafo(self):
    #Imprimir o grafo no console, tentem deixar próximo da representação 
    # dos slides (não precisa da grade da matriz).
    for valor in range(self.vertices):
      print(valor, ":", self.lista[valor])

  def inserirAresta(self, origem, destino, peso = 1):
    #Essa operação deve ter um cuidado especial,
    # ela deve ser executada levando em conta o tipo do grafo.
    # No caso de um grafo ponderado o peso deve ser aplicado e
    # no caso de um grafo direcionado, uma ligação de volta deve ser adicionada;
    
    #Converte os labels para indices
    if isinstance(origem, str):
      origem = self.labels.get(origem)
    if isinstance(destino, str):
      destino = self.labels.get(destino)
    #Se não ponderado, o peso é igual a 1
    if not self.ponderado:
      peso = 1
    
    self.lista[origem].append((destino, peso))

    if self.direcionado and origem != destino:
      self.lista[destino].append((origem, peso))

    self._ordernarArestasDoVertice

    self.arestas += 1

  def removerAresta(self, origem, destino):
    #Remove uma aresta entre dois vértices no grafo, lembrando que no grafo
    # não direcionado deve ser removida a aresta de retorno também;
    # try:
      #Converte os labels para indices
      if isinstance(origem, str):
        origem = self.labels.get(origem)
      if isinstance(destino, str):
        destino = self.labels.get(destino)

      for idx, aresta in enumerate(self.lista[origem]):
        if destino in aresta:
          self.lista[origem].pop(idx)

      # self.lista[origem].remove(self.lista[origem][destino])

      #No direcionado, remove o do destino, a origem
      if self.direcionado:
        for idx, aresta in enumerate(self.lista[destino]):
          if origem in aresta:
            self.lista[destino].pop(idx)
        # self.lista[destino].remove(self.lista[origem][destino])

      self._ordernarArestasDoVertice

      self.arestas -= 1
    # except:
    #   print('Aresta inválida.')

  def existeAresta(self, origem, destino):
    #Verifica a existência de uma aresta, aqui vemos uma diferença
    # grande entre matriz e lista.
    #Converte os labels para indices
    if isinstance(origem, str):
      origem = self.labels.get(origem)
    if isinstance(destino, str):
      destino = self.labels.get(destino)

    #Verifica se existe o destino na lista, ignorando o peso
    for aresta in self.lista[origem]:
      if destino in aresta:
        return True
      
    return False

  def pesoAresta(self, origem, destino):
    #Retorne o peso de uma aresta, aqui vemos uma diferença grande
    # entre matriz e lista.
    #Converte os labels para indices
    if isinstance(origem, str):
      origem = self.labels.get(origem)
    if isinstance(destino, str):
      destino = self.labels.get(destino)
    # print('\nPESOARESTA CHAMADO', end=' ')
    # print('origem:', origem, end=' ')
    # print('destino:', destino)

    # print('self.lista[origem]', self.lista[origem])
    # print('self.lista[origem][destino]', self.lista[origem][destino])
    # print('self.lista[origem][destino][1]', self.lista[origem][destino][1])
    return self.lista[origem][destino][1]

  def pesoArestaLabel(self, origem, destino):
    #Retorne o peso de uma aresta, aqui vemos uma diferença grande
    # entre matriz e lista.
    #Converte os labels para indices
    if isinstance(origem, str):
      origem = self.labels.get(origem)
    if isinstance(destino, str):
      destino = self.labels.get(destino)
    # print('\nPESOARESTA CHAMADO', end=' ')
    # print('origem:', origem, end=' ')
    # print('destino:', destino)

    # print('self.lista[origem]', self.lista[origem])
    # print('self.lista[origem][destino]', self.lista[origem][destino])
    # print('self.lista[origem][destino][1]', self.lista[origem][destino][1])
    #Verifica se existe o destino na lista, ignorando o peso
    for aresta in self.lista[origem]:
      # print('aresta:', aresta)
      # print('aresta:', aresta)
      # print('aresta[0]:', aresta[0])
      # print('destino:', aresta[0])
      if destino == aresta[0]:
        return aresta[1] 
    return None    

  def retornarVizinhos(self, vertice):
    # Converte para numero
    if isinstance(vertice, str):
      vertice = self.labels.get(vertice)
    #Função para retorno dos vizinhos de um vértice, necessária pois não
    # teremos acesso a estrutura das arestas para os próximos algoritmos.
    vizinhos = list()
    
    #Retorna cada tupla com os vizinhos
    for aresta in self.lista[vertice]:
      tuplaTemp = (self.labelVertice(aresta[0]), aresta[0])
      vizinhos.append(tuplaTemp)

    return vizinhos
  
  def temCicloTres(self):
    # Validar estrutura de dados dos vizinhos
    for verticeAtual in range(self.vertices):
      vizinhos = self.retornarVizinhos(verticeAtual)
      for vizinho in vizinhos:
        subVizinhos = self.retornarVizinhos(vizinho[1])
        for subVizinho in subVizinhos:
          if subVizinho in vizinhos:
            return True
    return False

  def planaridade(self):
    if self.vertices <= 2:  # 
      return True  # O Grafo é planar
    elif self.temCicloTres():  # Verificar se contém ciclo três
      # Validar funcionamento do código das arestas (adicionar/remover)
      if (self.arestas <= (3 * self.vertices) - 6):  # A ≤ 3V – 6
        return True  # Pode ser planar

    return False

  def grauVertice(self, vertice):
    #Converte os labels para indices
    if isinstance(vertice, str):
      vertice = self.labels.get(vertice)
    if isinstance(vertice, list): # Se já for uma lista, retorna o len dela
      return len(vertice)

    # print(vertice)
    # print(type(vertice))

    return len(self.lista[vertice])

  def labelsOrdenadosPorGrau(self):
    return sorted(self.labels, reverse=True, key=self.grauVertice)

  def insereCor(self, label, cor):
    #Converte os indices para labels
    if isinstance(label, int):
      label = list(self.labels.keys())[list(self.labels.values()).index(label)]

    #Define nome da vértice, para a cor
    self.listaColorida[label] = cor

  def consultaCor(self, vertice):
    #Converte os indices para labels
    if isinstance(vertice, int):
      vertice = list(self.labels.keys())[list(self.labels.values()).index(vertice)]
    return self.listaColorida.get(vertice)

  def welshOld(self):
    cor = self.cores
    for idx in self.labelsOrdenadosPorGrau():
      #A cada vértice
      for vizinho in self.retornarVizinhos(idx):
        # A cada vizinho
        if self.listaColorida.get(vizinho[0]) == cor:
          self.cores += 1
          cor = self.cores
          self.insereCor(idx, cor)
          break
      else:
        self.insereCor(idx, cor)

  def welsh(self):
    start_time = time.time()
    cor = self.cores
    first = True
    Processando = True
    while Processando:
      for idx in self.labelsOrdenadosPorGrau():
        if first:
          self.insereCor(idx, cor)
          first = False
          continue
        #A cada vértice
        for vizinho in self.retornarVizinhos(idx):
          # A cada vizinho
          # x = input("Pause")
          if self.listaColorida.get(vizinho[0]) == cor:
            break
          else:
            self.insereCor(idx, cor)
            continue
        else:
          self.insereCor(idx, cor)
      if len(self.listaColorida) == len(self.labels):
        Processando = False
        break
      else:
        self.cores += 1
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Elapsed time:", elapsed_time, "seconds")
    return self.cores

  def welshNew(self):
    start_time = time.time()
    cor = self.cores
    first = True
    Processando = True
    while Processando:
      for idx in self.labelsOrdenadosPorGrau():
        if first:
          self.insereCor(idx, cor)
          first = False
          continue
        #A cada vértice
        for vizinho in self.retornarVizinhos(idx):
          # A cada vizinho
          # x = input("Pause")
          if self.listaColorida.get(vizinho[0]) == cor:
            break
          else:
            self.insereCor(idx, cor)
            continue
        else:
          self.insereCor(idx, cor)
      if len(self.listaColorida) == len(self.labels):
        Processando = False
        break
      else:
        self.cores += 1
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Elapsed time:", elapsed_time, "seconds")
    return self.cores


In [87]:
# grafoUm = GrafoLista()
# grafoUm.openFile('/content/drive/MyDrive/Colab Arquivos (CSV TXT)/Trabalho M1 - Grafos (Dijkstra)/espacoaereo.txt')
# grafoUm.imprimeGrafo()
# entrada = 5
# dijkstra = grafoUm.dijkstra(entrada)
# for vertice in range(len(dijkstra)):
#     print("Distância do vertice", str(entrada), "para o vertice", str(vertice), "é", str(dijkstra[vertice]), sep=" ")
# print(grafoUm.buscaDijkstra(0, 2))

grafoDois = GrafoLista()
grafoDois.openFile('/content/drive/MyDrive/Colab Arquivos (CSV TXT)/Trabalho M1 - Grafos (Dijkstra)/buscas.txt')
grafoDois.imprimeGrafo()
# print(grafoDois.grauVertice(6))
# print(grafoDois.consultaCor(6))
# print(grafoDois.labelsOrdenadosPorGrau())
# print(grafoDois.planaridade())
print(grafoDois.welsh())
# print(grafoDois.buscaProfunda(0, 4))
# print(grafoDois.buscaLargura(0, 4))
# print(grafoDois.retornarVizinhos(0))

A saída de streaming foi truncada nas últimas 5000 linhas.
vizinho: ('4', 4)
self.listaColorida.get(vizinho[0]) == cor
IDX 6
Else: InsereCor: 6 0
IDX 0
vizinho: ('1', 1)
Else
vizinho: ('2', 2)
Else
vizinho: ('3', 3)
Else
Else: InsereCor: 0 0
IDX 2
vizinho: ('0', 0)
self.listaColorida.get(vizinho[0]) == cor
IDX 1
vizinho: ('0', 0)
self.listaColorida.get(vizinho[0]) == cor
IDX 3
vizinho: ('0', 0)
self.listaColorida.get(vizinho[0]) == cor
IDX 4
vizinho: ('2', 2)
Else
vizinho: ('5', 5)
self.listaColorida.get(vizinho[0]) == cor
IDX 5
vizinho: ('2', 2)
Else
vizinho: ('4', 4)
self.listaColorida.get(vizinho[0]) == cor
IDX 6
Else: InsereCor: 6 0
IDX 0
vizinho: ('1', 1)
Else
vizinho: ('2', 2)
Else
vizinho: ('3', 3)
Else
Else: InsereCor: 0 0
IDX 2
vizinho: ('0', 0)
self.listaColorida.get(vizinho[0]) == cor
IDX 1
vizinho: ('0', 0)
self.listaColorida.get(vizinho[0]) == cor
IDX 3
vizinho: ('0', 0)
self.listaColorida.get(vizinho[0]) == cor
IDX 4
vizinho: ('2', 2)
Else
vizinho: ('5', 5)
self.listaColo

KeyboardInterrupt: ignored

# GrafoLista com Cor

In [ ]:
from typing_extensions import Self
from operator import itemgetter
from queue import PriorityQueue
import time
class GrafoLista(Grafos):

  def __init__(self, direcionado = False, ponderado= False):
    super().__init__(direcionado, ponderado) # Direcionado = False significa que é direcionado, e True que não é Direcionado
    self.lista = list()
    self.labels = dict() #Contém os nomes de vértices

    self.vertices = 0
    self.arestas = 0
    self.cores = 0 

  def __str__(self):
    return (str(self.lista))

  def openFile(self, filename):
    if self.vertices != 0 or self.arestas != 0:
      raise Exception('A instância do objeto GrafoLista não está vazia.')

    with open(filename, 'r') as arquivo:
      verticesArquivo, arestasArquivo, direcionadoArquivo, ponderadoArquivo = (arquivo.readline()).split()
      #Checa se é direcionado
      if int(direcionadoArquivo) == 1:
        self.direcionado = False
      else:
        self.direcionado = True
      #Checa se é ponderado
      if int(ponderadoArquivo) == 1:
        self.ponderado = True
      else:
        self.ponderado = False

      for vertice in range(int(verticesArquivo)):
        self.inserirVertice(str(vertice))

      if self.ponderado:
        for aresta in range(int(arestasArquivo)):
          origem, destino, peso = (arquivo.readline()).split()
          self.inserirAresta(origem, destino, float(peso))
      else:
          for aresta in range(int(arestasArquivo)):
            origem, destino = (arquivo.readline()).split()
            self.inserirAresta(origem, destino)

  def buscaProfunda(self, origem, destino):
    caminho = []
    visitado = set()
    return self._buscaProfunda(origem, destino, caminho, visitado)

  def _buscaProfunda(self, origem, destino, caminho = [], visitado = set()):
    #Converte os labels para indices
    if isinstance(origem, str):
      origem = self.labels.get(origem)
    if isinstance(destino, str):
      destino = self.labels.get(destino)
    caminho.append(origem)
    visitado.add(origem)
    
    if origem == destino:
        return caminho
    for (vizinho, peso) in self.lista[origem]:
        if vizinho not in visitado:
            resultado = self._buscaProfunda(vizinho, destino, caminho, visitado)
            if resultado is not None:
                return resultado
    #caminho.pop() #diferença entre o visitado e o caminho, sem o pop, ele é igual a ordem visitada
    return None

  def buscaLargura(self, origem, destino, validaDestinoAntes = True, visitadoDebug = False):
    #Converte os labels para indices
    if isinstance(origem, str):
      origem = self.labels.get(origem)
    if isinstance(destino, str):
      destino = self.labels.get(destino)

    visitados = [origem]
    fila = [origem]

    while len(fila) > 0:
      # print('Laço de repetição!')
      verticeAtual = fila.pop()

      if verticeAtual == destino:
        # print('verticeAtual == destino' + str(verticeAtual) + str(destino), sep=';')
        if verticeAtual not in visitados:
          visitados.append(verticeAtual)
        return visitados

      vizinhos = self.retornarVizinhos(verticeAtual)
      # print('VerticeAtual: ' + str(verticeAtual), end=' ')
      # print('Vizinhos: ' + str(vizinhos))

      for verticeVizinha in vizinhos:
        idxVerticeVizinha = self.labels.get(verticeVizinha[0])
        # print('idxVerticeVizinha: ' + str(idxVerticeVizinha))
        if idxVerticeVizinha not in visitados:
          fila.append(idxVerticeVizinha)
          visitados.append(idxVerticeVizinha)
          if idxVerticeVizinha == destino and validaDestinoAntes:
            return visitados
      
      # print('Fila: ' + str(fila), end=' ')
      # print('Visitados: ' + str(visitados))
      # print('Length Fila: ' + str(len(fila)))
      
    if visitadoDebug:
      return visitados
    return None

  def buscaDijkstra(self, origem, destino):
    #Converte os labels para indices
    if isinstance(origem, str):
      origem = self.labels.get(origem)
    if isinstance(destino, str):
      destino = self.labels.get(destino)

    caminhosMaisCurtos = self.dijkstra(origem)
    return caminhosMaisCurtos[destino]

  def dijkstra(self, origem):
      #Converte os labels para indices
      if isinstance(origem, str):
        origem = self.labels.get(origem)

      caminhosMaisCurtos = {vertice:float('inf') for vertice in range(self.vertices)}
      caminhosMaisCurtos[origem] = 0
      # print('caminhosMaisCurtos inicial:', caminhosMaisCurtos)

      fila = PriorityQueue()
      fila.put((0, origem))
      # print('Fila inicial:', fila.queue)
      visitado = list()

      while not fila.empty():
          (distanciaAtual, verticeAtual) = fila.get()
          visitado.append(verticeAtual)

          vizinhos = self.retornarVizinhos(verticeAtual)
          # print('Vizinhos:', vizinhos)
          # print('Fila loop:', fila.queue)
          for vizinho in vizinhos:
            # print('verticeAtual:', verticeAtual, end=' ')
            # print('VizinhoAtual:', vizinho, end=' ')
            distanciaVizinho = self.pesoArestaLabel(verticeAtual, vizinho[1])
            # print('distanciaVizinhoAtual:', distanciaVizinho)
            if vizinho not in visitado:
              custoAntigo = caminhosMaisCurtos[self.labels.get(vizinho[0])]
              custoNovo = caminhosMaisCurtos[verticeAtual] + distanciaVizinho
              if custoNovo < custoAntigo:
                fila.put((custoNovo, self.labels.get(vizinho[0])))
                caminhosMaisCurtos[self.labels.get(vizinho[0])] = custoNovo

      return caminhosMaisCurtos

  def _ordernarArestasDoVertice(self):
    for verticeId in range(len(self.lista)):
      self.lista[verticeId].sort(key=itemgetter(0))

  def inserirVertice(self, label):
    #Adiciona um vértice sem nenhuma aresta associada a ele, pode parecer 
    # igual em ambos os casos, mas não é.
    #Precisamos adicionar esse vértice no vetor de vértices e também alocar
    # seu espaço para as arestas.

    #Define nome da vértice
    self.labels[label] = self.vertices
    self.vertices += 1

    self.lista.append((list(), 0))
  
  def removerVertice(self, label):
    #Remove um vértice do grafo, elimina a linha e coluna dele da matriz 
    # e a referência dele da lista, junto com todas as arestas que chegam 
    # e saem dele.
    #Converte o indice para label
    if isinstance(label, int):
      label = self.labelVertice(label)

    #Removendo o label
    idVertice = self.labels.get(label)
    self.labels.pop(label)
    #Balanceando os labels
    labelsTmp = dict()
    count = 0
    for label in self.labels.items():
      labelsTmp[label[0]] = count
      count += 1
    self.labels = labelsTmp
    #Diminuindo o número de vértices
    self.vertices -= 1

    listaTmp = list()
    for vertice in range(self.vertices + 1):
      if vertice == idVertice:
        continue
      listaTmp.append(self.lista[vertice])

    self.lista = listaTmp
  
  def labelVertice(self, indice):
    #Funções básicas para retornar o nome de um vértice.
    label = [k for k, v in self.labels.items() if v == indice]
    return (label[0])

  def imprimeGrafo(self):
    #Imprimir o grafo no console, tentem deixar próximo da representação 
    # dos slides (não precisa da grade da matriz).
    for valor in range(self.vertices):
      print(valor, ":", self.lista[valor])

  def inserirAresta(self, origem, destino, peso = 1):
    #Essa operação deve ter um cuidado especial,
    # ela deve ser executada levando em conta o tipo do grafo.
    # No caso de um grafo ponderado o peso deve ser aplicado e
    # no caso de um grafo direcionado, uma ligação de volta deve ser adicionada;
    
    #Converte os labels para indices
    if isinstance(origem, str):
      origem = self.labels.get(origem)
    if isinstance(destino, str):
      destino = self.labels.get(destino)
    #Se não ponderado, o peso é igual a 1
    if not self.ponderado:
      peso = 1
    
    self.lista[origem].append((destino, peso))

    if self.direcionado and origem != destino:
      self.lista[destino].append((origem, peso))

    self._ordernarArestasDoVertice

    self.arestas += 1

  def removerAresta(self, origem, destino):
    #Remove uma aresta entre dois vértices no grafo, lembrando que no grafo
    # não direcionado deve ser removida a aresta de retorno também;
    # try:
      #Converte os labels para indices
      if isinstance(origem, str):
        origem = self.labels.get(origem)
      if isinstance(destino, str):
        destino = self.labels.get(destino)

      for idx, aresta in enumerate(self.lista[origem]):
        if destino in aresta:
          self.lista[origem].pop(idx)

      # self.lista[origem].remove(self.lista[origem][destino])

      #No direcionado, remove o do destino, a origem
      if self.direcionado:
        for idx, aresta in enumerate(self.lista[destino]):
          if origem in aresta:
            self.lista[destino].pop(idx)
        # self.lista[destino].remove(self.lista[origem][destino])

      self._ordernarArestasDoVertice

      self.arestas -= 1
    # except:
    #   print('Aresta inválida.')

  def existeAresta(self, origem, destino):
    #Verifica a existência de uma aresta, aqui vemos uma diferença
    # grande entre matriz e lista.
    #Converte os labels para indices
    if isinstance(origem, str):
      origem = self.labels.get(origem)
    if isinstance(destino, str):
      destino = self.labels.get(destino)

    #Verifica se existe o destino na lista, ignorando o peso
    for aresta in self.lista[origem]:
      if destino in aresta:
        return True
      
    return False

  def pesoAresta(self, origem, destino):
    #Retorne o peso de uma aresta, aqui vemos uma diferença grande
    # entre matriz e lista.
    #Converte os labels para indices
    if isinstance(origem, str):
      origem = self.labels.get(origem)
    if isinstance(destino, str):
      destino = self.labels.get(destino)
    # print('\nPESOARESTA CHAMADO', end=' ')
    # print('origem:', origem, end=' ')
    # print('destino:', destino)

    # print('self.lista[origem]', self.lista[origem])
    # print('self.lista[origem][destino]', self.lista[origem][destino])
    # print('self.lista[origem][destino][1]', self.lista[origem][destino][1])
    return self.lista[origem][destino][1]

  def pesoArestaLabel(self, origem, destino):
    #Retorne o peso de uma aresta, aqui vemos uma diferença grande
    # entre matriz e lista.
    #Converte os labels para indices
    if isinstance(origem, str):
      origem = self.labels.get(origem)
    if isinstance(destino, str):
      destino = self.labels.get(destino)
    # print('\nPESOARESTA CHAMADO', end=' ')
    # print('origem:', origem, end=' ')
    # print('destino:', destino)

    # print('self.lista[origem]', self.lista[origem])
    # print('self.lista[origem][destino]', self.lista[origem][destino])
    # print('self.lista[origem][destino][1]', self.lista[origem][destino][1])
    #Verifica se existe o destino na lista, ignorando o peso
    for aresta in self.lista[origem]:
      # print('aresta:', aresta)
      # print('aresta:', aresta)
      # print('aresta[0]:', aresta[0])
      # print('destino:', aresta[0])
      if destino == aresta[0]:
        return aresta[1] 
    return None    

  def retornarVizinhos(self, vertice):
    # Converte para numero
    if isinstance(vertice, str):
      vertice = self.labels.get(vertice)
    #Função para retorno dos vizinhos de um vértice, necessária pois não
    # teremos acesso a estrutura das arestas para os próximos algoritmos.
    vizinhos = list()
    
    #Retorna cada tupla com os vizinhos
    for aresta in self.lista[vertice]:
      tuplaTemp = (self.labelVertice(aresta[0]), aresta[0])
      vizinhos.append(tuplaTemp)

    return vizinhos
  
  def temCicloTres(self):
    # Validar estrutura de dados dos vizinhos
    for verticeAtual in range(self.vertices):
      vizinhos = self.retornarVizinhos(verticeAtual)
      for vizinho in vizinhos:
        subVizinhos = self.retornarVizinhos(vizinho[1])
        for subVizinho in subVizinhos:
          if subVizinho in vizinhos:
            return True
    return False

  def planaridade(self):
    if self.vertices <= 2:  # 
      return True  # O Grafo é planar
    elif self.temCicloTres():  # Verificar se contém ciclo três
      # Validar funcionamento do código das arestas (adicionar/remover)
      if (self.arestas <= (3 * self.vertices) - 6):  # A ≤ 3V – 6
        return True  # Pode ser planar

    return False

  def grauVertice(self, vertice):
    #Converte os labels para indices
    if isinstance(vertice, str):
      vertice = self.labels.get(vertice)
    if isinstance(vertice, list): # Se já for uma lista, retorna o len dela
      return len(vertice)

    # print(vertice)
    # print(type(vertice))

    return len(self.lista[vertice])

  def labelsOrdenadosPorGrau(self):
    return sorted(self.labels, reverse=True, key=self.grauVertice)

  def insereCor(self, label, cor):
    #Converte os indices para labels
    if isinstance(label, int):
      label = list(self.labels.keys())[list(self.labels.values()).index(label)]

    #Define nome da vértice, para a cor
    self.listaColorida[label] = cor

  def consultaCor(self, vertice):
    #Converte os indices para labels
    if isinstance(vertice, int):
      vertice = list(self.labels.keys())[list(self.labels.values()).index(vertice)]
    return self.listaColorida.get(vertice)

  def welshOld(self):
    cor = self.cores
    for idx in self.labelsOrdenadosPorGrau():
      #A cada vértice
      for vizinho in self.retornarVizinhos(idx):
        # A cada vizinho
        if self.listaColorida.get(vizinho[0]) == cor:
          self.cores += 1
          cor = self.cores
          self.insereCor(idx, cor)
          break
      else:
        self.insereCor(idx, cor)

  def welsh(self):
    start_time = time.time()
    cor = self.cores
    first = True
    Processando = True
    while Processando:
      for idx in self.labelsOrdenadosPorGrau():
        if first:
          self.insereCor(idx, cor)
          first = False
          continue
        #A cada vértice
        for vizinho in self.retornarVizinhos(idx):
          # A cada vizinho
          # x = input("Pause")
          if self.listaColorida.get(vizinho[0]) == cor:
            break
          else:
            self.insereCor(idx, cor)
            continue
        else:
          self.insereCor(idx, cor)
      if len(self.listaColorida) == len(self.labels):
        Processando = False
        break
      else:
        self.cores += 1
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Elapsed time:", elapsed_time, "seconds")
    return self.cores

  def welshNew(self):
    start_time = time.time()
    cor = self.cores
    first = True
    Processando = True
    while Processando:
      for idx in self.labelsOrdenadosPorGrau():
        if first:
          self.insereCor(idx, cor)
          first = False
          continue
        #A cada vértice
        for vizinho in self.retornarVizinhos(idx):
          # A cada vizinho
          # x = input("Pause")
          if self.listaColorida.get(vizinho[0]) == cor:
            break
          else:
            self.insereCor(idx, cor)
            continue
        else:
          self.insereCor(idx, cor)
      if len(self.listaColorida) == len(self.labels):
        Processando = False
        break
      else:
        self.cores += 1
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Elapsed time:", elapsed_time, "seconds")
    return self.cores
